## Table of Contents
- [Convolutional Neural Network](#Convolutional-Neural-Network)
  - [Import Libraries](#Import-Libraries)
  - [Load data](#Load-data)
  - [Data Description](#Data-Description)
  - [Model Preparation](#Model-Preparation)
  - [Model Preparation](#Model-Preparation)
  - [Model Summary](#Model-Summary)
  - [Model Training](#Model-Training)
- [🏠 Home](../../../../welcomePage.ipynb)

# Convolutional Neural Network

Convolutional Neural Networks (CNNs) are a class of deep neural networks primarily used for processing and analyzing visual data. They are particularly effective in tasks involving images due to their ability to capture spatial hierarchies and learn intricate patterns.

### Key Components of CNNs:

1. **Convolutional Layers:** These layers apply convolution operations using learnable filters that slide over the input image, capturing local patterns and features.

2. **Pooling Layers:** Pooling layers reduce the spatial dimensions of the feature maps produced by convolutional layers, while preserving important information.

3. **Activation Functions:** Typically, ReLU (Rectified Linear Unit) is used to introduce non-linearity after convolution and pooling operations.

4. **Fully Connected Layers:** These layers take the high-level features extracted by convolutional layers and use them for classification or regression tasks.

### Applications of CNNs:

- **Image Classification:** Assigning labels to images based on their content.
  
- **Object Detection:** Locating and classifying objects within images.

- **Image Segmentation:** Dividing images into meaningful parts or objects.
  
- **Face Recognition:** Identifying and verifying individuals based on facial features.

- **Medical Image Analysis:** Analyzing medical images for diagnostic purposes, such as detecting tumors or abnormalities.

- **Video Analysis:** Understanding and processing video content, such as action recognition or tracking objects over time.

### Why CNNs are Effective:

CNNs leverage parameter sharing and spatial locality to efficiently learn and generalize from visual data. This makes them well-suited for tasks where the arrangement of pixels and features in images is crucial for accurate analysis.

In this notebook, we'll implement and explore the application of CNNs for a specific task, demonstrating their effectiveness in handling complex visual data.


## Import Libraries

**Press ▶ to import the libraries.**

In [ ]:
# Imports
print("Importing liblease wait...")
import matplotlib.pyplot as plt
import numpy as np
import PIL
import os
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from IPython.display import display
import ipywidgets as widgets
from IPython.display import display, Video

print("All libraries were imported successfully.")

## Load data

**Press ▶ to load and display the data.**

In [ ]:
import os
from PIL import Image
import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display, clear_output,HTML

# Create a FileChooser widget for selecting a folder
folder_chooser = FileChooser()
folder_chooser.show_only_dirs = True

# Create a "Select" button widget
select_button = widgets.Button(
    description='Select',
    disabled=False,
    button_style='',
    tooltip='Click to select folder',
    icon='check'
)

# Create a "Show Data" button widget
show_data_button = widgets.Button(
    description='Show Data',
    disabled=True,  # Initially disabled
    button_style='',
    tooltip='Click to show data',
    icon='eye'
)

# Output widget to display messages and images
output = widgets.Output()
image_display = widgets.Output()

# Initialize the stored_files array
stored_files = []

# Function to handle "Select" button click
def on_select_button_click(b):
    global selected_folder
    with output:
        clear_output()
        selected_folder = folder_chooser.selected
        if not selected_folder:
            print("No folder selected. Please select a folder.")
            return

        global stored_files
        stored_files = [os.path.join(selected_folder, f) for f in os.listdir(selected_folder) if os.path.isdir(os.path.join(selected_folder, f))]
        show_data_button.disabled = False if stored_files else True
        print(f"Folders from '{selected_folder}' have been loaded.")

# Function to display one image from each folder
def on_show_data_button_click(b):
    with image_display:
        clear_output(wait=True)
        for folder in stored_files:
            folder_name = os.path.basename(folder)
            for file_name in os.listdir(folder):
                if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    img_path = os.path.join(folder, file_name)
                    image = Image.open(img_path)
                    # Resize image and add title
                    image = image.resize((300, 300))  # Resize the image to 300x300 pixels
                    display(HTML(f"<h3>{folder_name}</h3>"))  # Display the folder name as a title
                    display(image)
                    break  # Display only one image per folder

# Attach the functions to the button widgets
select_button.on_click(on_select_button_click)
show_data_button.on_click(on_show_data_button_click)

# Display the folder chooser, buttons, and output widgets
display(folder_chooser)
display(select_button)
display(show_data_button)
display(output)
display(image_display)


**Press ▶ to register the data.**

In [ ]:
data_dir = selected_folder
print ("Data is registered successfully.")

## Data Description

The dataset was collected with a RPI and a Ender 3 pro in order to detect the anomalies during the printing process. It consists of 759 defected print images and 798 non-defected print images.

**Press ▶ to display the number of images per class.**

In [ ]:
def count_files_in_folders(data_dir):
    for root, dirs, files in os.walk(data_dir):
        if root == data_dir:
            continue  # Skip the starting directory itself
        folder_name = os.path.relpath(root, data_dir)
        file_count = len(files)
        print(f"Folder '{folder_name}' contains {file_count} files")

count_files_in_folders(data_dir)


## Model Preparation

In the context of training a Convolutional Neural Network (CNN) in frameworks like TensorFlow or Keras, specifying the batch size, image height, and image width is crucial for several reasons:

**Batch Size:**
- **Training Efficiency:** Specifying a batch size determines how many samples (images and their corresponding labels) are propagated through the network at once during each training iteration.
- **Memory Management:** It helps in managing memory usage, as processing too many images at once might exceed available memory, while too few may not fully utilize hardware resources.

**Image Dimensions (Height and Width):**
- **Input Shape:** CNNs expect input data to have a consistent shape. Specifying image height and width ensures that all images in the dataset are resized or processed to a uniform size before being fed into the network.
- **Convolutional Layer Requirements:** Convolutional layers in CNNs require input images to have a defined shape (height, width, number of channels), which is specified in the first layer of the model (e.g., `input_shape=(height, width, channels)`).

**Model Architecture Requirements:**
- **Dimension Matching:** Each layer in the CNN expects input data in a specific format (e.g., images as matrices of pixel values). Specifying dimensions ensures that the data fed into the network matches these expectations.
- **Performance and Accuracy:** Correctly sized input images and appropriate batch sizes help in achieving optimal performance and accuracy during training and inference.


**Press ▶ to specify the batch size, image height, and the image width.**

In [ ]:
# Define global variables
batch_size = None
img_height = None
img_width = None

# Function to handle button click event
def on_button_click(b):
    global batch_size, img_height, img_width
    batch_size_input = text_box_batch_size.value.strip()
    img_height_input = text_box_img_height.value.strip()
    img_width_input = text_box_img_width.value.strip()
    
    if batch_size_input == '' or img_height_input == '' or img_width_input == '':
        with output_widget:
            output_widget.clear_output()
            print("Error: Please enter values for all fields.")
    else:
        try:
            batch_size = int(batch_size_input)
            img_height = int(img_height_input)
            img_width = int(img_width_input)
            output_widget.clear_output()
            with output_widget:
                print(f"Variables set successfully: batch_size={batch_size}, img_height={img_height}, img_width={img_width}")
        except ValueError:
            with output_widget:
                output_widget.clear_output()
                print("Please enter valid integers.")

# Create widgets
text_box_batch_size = widgets.Text(placeholder='Enter batch size', description='Batch Size:',layout=widgets.Layout(width='300px'), style={'description_width': '150px'})
text_box_img_height = widgets.Text(placeholder='Enter image height', description='Image Height:',layout=widgets.Layout(width='300px'), style={'description_width': '150px'})
text_box_img_width = widgets.Text(placeholder='Enter image width', description='Image Width:',layout=widgets.Layout(width='300px'), style={'description_width': '150px'})
button_set_variables = widgets.Button(description='Set Variables')
output_widget = widgets.Output()

# Assign button click event
button_set_variables.on_click(on_button_click)

# Display widgets
display(text_box_batch_size, text_box_img_height, text_box_img_width, button_set_variables, output_widget)


**Press ▶ to split the data into training and testing sets.**

In [ ]:
# Define global variables
validation_split = None

# Function to handle button click event
def on_button_click(b):
    global validation_split
    validation_split = slider.value / 100.0  # Convert slider value to a fraction
    output_widget.clear_output()
    with output_widget:
        print(f"Validation percentage set to: {validation_split:.0%}");

# Create widgets
slider = widgets.IntSlider(value=20, min=0, max=90, step=1, description='Validation Split %:',layout=widgets.Layout(width='500px'), style={'description_width': '150px'})
button_set_split = widgets.Button(description='Set Split')
output_widget = widgets.Output()

# Assign button click event
button_set_split.on_click(on_button_click)

# Display widgets
display(slider, button_set_split, output_widget)


**Press ▶ to display the number of images in the training set.**

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory = data_dir,
    validation_split=validation_split,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

**Press ▶ to display the number of images in the testing sets.**

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory = data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

**Press ▶ to display the classes' names.**

In [ ]:
class_names = train_ds.class_names
print("The data contains classes:")
print(class_names)

**Press ▶ to display some of the images.**

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

## Model Preparation

**Press ▶ to maximize model performance.**

In [ ]:
# Training and validation data preparation for improved performance
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

print("Training and validation datasets processed for performance maximization.")

**Press ▶ to normalize the data.**

In [ ]:
normalization_layer = layers.Rescaling(1./255.0)

normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
## Notice the pixel values are now in `[0,1]`.
#print(np.min(first_image), np.max(first_image)) # To check if data was normalized correctly

print ("Training images data was normalized.")

**Press ▶ to define the model.**

In [ ]:
num_classes = len(class_names)

model = Sequential([
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes)
])

print ("Model has been defined.")

**Press ▶ to compile the model.**

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

print ("Model has been compiled.")

## Model Summary

**Press ▶ to display the model summary.**

In [ ]:
model.summary()

## Model Training

**Press ▶ to train the model.**

In [ ]:
epochs=15
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

print('')
print('Model has been successfully trained.')

### Validation and Visualization

**Press ▶ to show the accuracies and the losses of the model.**

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 5))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

**Press ▶ to display the test accuracy and the test loss values.**

In [ ]:
# verify the performance of the model
loss, accuracy = model.evaluate(val_ds, verbose=0)
print('Test accuracy :', accuracy)
print('Test loss:', loss)

**Press ▶ to display instances of the testing set predictions.**

In [ ]:
# get a batch from validation_ds to do some inference
image_batch, label_batch = val_ds.as_numpy_iterator().next()

# inference
inference = model.predict_on_batch(image_batch)

# apply softmax to convert logits to probabilities
probabilities = tf.nn.softmax(inference, axis=-1).numpy()

# show imgs and labels
plt.figure(figsize=(18, 18))
for i in range(12):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(image_batch[i].astype("uint8"))
    plt.title('Inference:{}, {:.0f}% Confidence\nReal Label:{}'
              .format(class_names[np.argmax(probabilities[i])], 100 * np.max(probabilities[i]), class_names[label_batch[i]]))
    plt.axis("off")


### <center>[🏠 Home](../../../../welcomePage.ipynb)</center>